# Initialization

to run on a different computer, run ``jupyter notebook --ip='*'`` copy and paste address and replace ``localhost`` with ``jmfranck-pi2.syr.edu`` (if on the internet) or ``192.168.1.20`` (if on local network)

(test:make change to see if file commits to github)

In [ ]:
%pylab inline
from serial.tools.list_ports import comports, grep
import serial
import time
#import winsound
print serial.VERSION

def generate_beep(freq,duration):
    "Generates a beep -- used winsound.Beep under windows -- use pygame under linux"
    return
def bridge12_wait(s):
    #time.sleep(5)
    def look_for(this_str):
        for j in range(1000):
            a = s.read_until(this_str+'\r\n')
            time.sleep(0.1)
            #print "a is",repr(a)
            print "look for",this_str,"try",j+1
            if this_str in a:
                print "found: ",this_str
                break
    look_for('MPS Started')
    look_for('System Ready')
    return

Print out the com ports, so we know what we're looking for

In [ ]:
[(j.device, j.hwid, j.vid, j.description, j.manufacturer) for j in comports()] #list comprehension

Grab the port labeled as Arduino (since the Bridge12 microcontroller is an Arduino)

In [ ]:
portlist = [j.device for j in comports() if u'Arduino Due' in j.description]
assert len(portlist)==1
thisport = portlist[0]

### Testing Out Commands

Help Command

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s) #give it time to start
    s.write("help\r") #command for "help"
    print "after help:"
    entire_response = ''
    start = time.time()
    entire_response += s.readline()
    time_to_read_a_line = time.time()-start
    grab_more_lines = True
    while grab_more_lines:
        time.sleep(3*time_to_read_a_line)
        more = s.read_all()
        entire_response += more
        if len(more) == 0:
            grab_more_lines = False
    print repr(entire_response)

Waveguide Status Command

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    def wgstatus_int_singletry():
        s.write('wgstatus?\r')
        return int(s.readline())
    def wgstatus_int():
        "need two consecutive responses that match"
        a = wgstatus_int_singletry()
        b = wgstatus_int_singletry()
        while a != b:
            a = b
            b = wgstatus_int_singletry()
        return a
    bridge12_wait(s)
    print "initial status:",wgstatus_int()
    s.write('wgstatus 1\r')
    print "turn it on:",wgstatus_int()
    time.sleep(3)
    s.write('wgstatus 0\r')
    print "turn it off again:",wgstatus_int()

Waveguide Status Command 2

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    def set_wg(setting):
        """set *and check* waveguide
        
        Parameters
        ==========
        setting: bool
            Waveguide setting appropriate for:
            True: DNP
            False: ESR
        """
        s.write('wgstatus %d\r'%setting)
        for j in range(10):
            s.write('wgstatus?\r')
            result = int(s.readline())
            if result == setting:
                return
        raise RuntimeError("After trying 10 times, I can't get the waveguide to change")
    print "waveguide on"
    set_wg(True)
    time.sleep(5)# just so we're not toggling it back and forth very fast
    print "waveguide off"
    set_wg(False)

Frequency Query Test

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s) #give it time to start
    s.write("freq?\r") #query for "freq?"
    print "after freq:"
    entire_response = ''
    start = time.time()
    entire_response += s.readline()
    time_to_read_a_line = time.time()-start
    grab_more_lines = True
    while grab_more_lines:
        time.sleep(3*time_to_read_a_line)
        more = s.read_all()
        entire_response += more
        if len(more) == 0:
            grab_more_lines = False
    print repr(entire_response)

Frequency Sweep Test

In [ ]:
freq = linspace(9.4e9,9.9e9, 50)
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s) #give it time to start
    for j,f in enumerate(freq):
        time.sleep(0.5)
        generate_beep(500, 300)
        s.write('freq %.1f\r'%(f/1e3))
    print "after freq:"
    entire_response = ''
    start = time.time()
    entire_response += s.readline()
    time_to_read_a_line = time.time()-start
    grab_more_lines = True
    while grab_more_lines:
        time.sleep(3*time_to_read_a_line)
        more = s.read_all()
        entire_response += more
        if len(more) == 0:
            grab_more_lines = False
    print repr(entire_response)

Ampstatus Test

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    def ampstatus_int_singletry():
        s.write('ampstatus?\r')
        return int(s.readline())
    def ampstatus_int():
        "need two consecutive responses that match"
        a = ampstatus_int_singletry()
        b = ampstatus_int_singletry()
        while a != b:
            a = b
            b = wgstatus_int_singletry()
        return a
    bridge12_wait(s)
    print "initial status:",wgstatus_int()
    s.write('ampstatus 1\r')
    print "turn it on:",wgstatus_int()
    time.sleep(3)
    s.write('ampstatus 0\r')
    print "turn it off again:",wgstatus_int()

Power Change Test

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    def power_int_singletry():
        s.write('power?\r')
        return int(s.readline())
    def power_int():
        "need two consecutive responses that match"
        a = power_int_singletry()
        b = power_int_singletry()
        while a != b:
            a = b
            b = power_int_singletry()
        return a
    bridge12_wait(s)
    print "initial power:",power_int()
    s.write('power 100\r')
    print "up power:",power_int()
    time.sleep(3)
    s.write('power 0\r')
    print "down power:",power_int()

### Initialization Part 2
###### Initialization code for Tuning Curve and MW Power Test

In [ ]:
def wgstatus_int_singletry(s):
    s.write('wgstatus?\r')
    return int(s.readline())
def wgstatus_int(s):
    "need two consecutive responses that match"
    c = wgstatus_int_singletry(s)
    d = wgstatus_int_singletry(s)
    while c != d:
        c = d
        d = wgstatus_int_singletry(s)
    return c
def set_wg(s,setting):
    """set *and check* waveguide

    Parameters
    ==========
    s: Serial object
        gives the connection
    setting: bool
        Waveguide setting appropriate for:
        True: DNP
        False: ESR
    """
    s.write('wgstatus %d\r'%setting)
    for j in range(10):
        result = wgstatus_int(s)
        if result == setting:
            return
    raise RuntimeError("After checking status 10 times, I can't get the waveguide to change")
    
def ampstatus_int_singletry(s):
    s.write('ampstatus?\r')
    return int(s.readline())
def ampstatus_int(s):
    "need two consecutive responses that match"
    a = ampstatus_int_singletry(s)
    b = ampstatus_int_singletry(s)
    while a != b:
        a = b
        b = ampstatus_int_singletry(s)
    return a
def set_amp(s,setting):
    """set *and check* amplifier

    Parameters
    ==========
    s: Serial object
        gives the connection
    setting: bool
       Amplifier setting appropriate for:
        True: On
        False: Off
    """
    s.write('ampstatus %d\r'%setting)
    for j in range(10):
        result = ampstatus_int(s)
        if result == setting:
            return
    raise RuntimeError("After checking status 10 times, I can't get the amplifier to turn on/off")
    
def rfstatus_int_singletry(s):
    s.write('rfstatus?\r')
    return int(s.readline())
def rfstatus_int(s):
    "need two consecutive responses that match"
    f = rfstatus_int_singletry(s)
    g = rfstatus_int_singletry(s)
    while f != g:
        f = g
        g = rfstatus_int_singletry(s)
    return f
def set_rf(s,setting):
    """set *and check* microwave power

    Parameters
    ==========
    s: Serial object
        gives the connection
    setting: bool
        Microwave Power setting appropriate for:
        True: On
        False: Off
    """
    s.write('rfstatus %d\r'%setting)
    for j in range(10):
        result = rfstatus_int(s)
        if result == setting:
            return
    raise RuntimeError("After checking status 10 times, I can't get the mw power to turn on/off")

def power_int_singletry(s):
    s.write('power?\r')
    return int(s.readline())
def power_int(s):
    "need two consecutive responses that match"
    h = power_int_singletry(s)
    i = power_int_singletry(s)
    while h != i:
        h = i
        i = power_int_singletry(s)
    return h
def set_power(s,dBm):
    """set *and check* power

    Parameters
    ==========
    s: Serial object
        gives the connection
    dBm: float
        power values -- give a dBm (not 10*dBm) as a floating point number
    """
    setting = int(10*dBm+0.5)
    if setting > 150:
        raise ValueError("You are not allowed to use this function to set a power of greater than 15 dBm for safety reasons")
    elif setting < 0:
        raise ValueError("Negative dBm -- not supported")
    s.write('power %d\r'%setting)
    for j in range(10):
        result = power_int(s)
        if result == setting:
            return
    raise RuntimeError("After checking status 10 times, I can't get the power to change")
    

###### Code for MW Power Test
check that all the ``set_`` functions work correctly:

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    print "initial wg status:", wgstatus_int(s)
    set_wg(s,True)
    print "initial amp status:", ampstatus_int(s)
    set_amp(s, True)
    print "initial rf status:", rfstatus_int(s)
    set_rf(s, True)
    print "initial power:",power_int(s)
    set_power(s, 10)
    print "up power:",power_int(s)
    time.sleep(10)
    set_power(s, 0)
    print "down power:",power_int(s)
    set_rf(s,False)
    print "final rf status:", rfstatus_int(s)
    set_wg(s,False)
    print "final wg status:", wgstatus_int(s)

### Tuning Curve

In [ ]:
freq = linspace(9.4e9,9.9e9, 50)
def freq_sweep(s,freq):
    """sweep over an array of frequencies

    Parameters
    ==========
    s: Serial object
        gives the connection
    freq: array of floats
        frequencies in Hz
        
    Returns
    =======
    rxvalues: array
        An array of floats, same length as freq, containing the receiver (reflected) power in dBm at each frequency.
    """    
    rxvalues = zeros(len(freq))
    #FREQUENCY AND RXPOWER SWEEP
    for j,f in enumerate(freq):
            time.sleep(0.5)
            generate_beep(500, 300)
            s.write('freq %.1f\r'%(f/1e3))
            time.sleep(0.1)
            s.write('rxpowerdbm?\r') 
            rx =s.readline()
            rxvalues[j] = float(rx)/10 
    return rxvalues
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    #Turn Components On and Power to 10dBm
    set_wg(s,True)
    set_amp(s, True)
    set_rf(s, True)
    set_power(s,5.0)
    for j in range(1):
        rxvalues = freq_sweep(s,freq)
        plot(freq, rxvalues, alpha=0.5) #tuning curve
    #Turn Components Off and Power to 0dBm
    set_power(s,0)
    set_rf(s,False)
    set_wg(s,False)

Testing Components of Tuning Curve Individually (Ctrl + / to comment out)

In [ ]:
# with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
#     bridge12_wait(s)
#     print "initial wg status:",wgstatus_int(s)
#     set_wg(s,True)
#     time.sleep(10)
#     set_wg(s,False)
#     print "final wg status:", wgstatus_int(s)
# with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
#     bridge12_wait(s)
#     print "initial amp status:",ampstatus_int(s)
#     set_amp(s,True)
#     time.sleep(10)
#     set_amp(s,False)
#     print "final amp status:", ampstatus_int(s)
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    print "initial wg status:",wgstatus_int(s)
    set_wg(s,True)
    print "initial rf status:",ampstatus_int(s)
    set_rf(s,True)
    time.sleep(10)
    set_rf(s,False)
    print "final rf status:", rfstatus_int(s)
    set_wg(s,False)
    print "final wg status:", wgstatus_int(s)

There is an issue with ampstatus: the initial status is 1, which seems to be problematic. When I set it to True then False, the final amp status is 0.

The same issue appears with the rfstatus, but in this case, the Bridge12 screen says the mw power is off but the "initial rf status" reads it as on.